In [1]:
from chebGreen.greenlearning.model import *
from chebGreen.chebpy2 import Chebfun2, cheb2prefs
import matplotlib.pyplot as plt

%env TF_MLC_LOGGING=1
%env TF_CPP_MIN_LOG_LEVEL=2

env: TF_MLC_LOGGING=1
env: TF_CPP_MIN_LOG_LEVEL=2


In [2]:
model = GreenNN()
model.build(loadPath = "savedModels/laplace")

Metal device set to: Apple M1 Pro


In [3]:
%%time
g = Chebfun2(model.evaluateG, domain = [0, 1, 0, 1], prefs = cheb2prefs, simplify = False)

CPU times: user 521 ms, sys: 117 ms, total: 637 ms
Wall time: 526 ms


In [4]:
g

chebfun2 object
     domain       rank               corner values
[0,1] x [0,1]     46       [0.005 0.002 -0.001 0.002]
vertical scale = 0.005244

In [ ]:
u,s,v = g.svd()

In [ ]:
i = 30
fig = plt.figure(figsize = (8,6))
u[:,i].plot()
plt.title(f'{i+1}th Left singular function for a Laplacian')
plt.xlim([0,1])
# fig.savefig(f'Laplace-{i+1}.png', dpi = fig.dpi)

In [ ]:
xx = np.linspace(0,1,2000)
yy = np.linspace(0,1,2000)
x, y = np.meshgrid(xx,yy)

In [ ]:
%%time
Gneural = model.evaluateG(x,y)

# plt.contourf(x,y,Gneural, 50, cmap = 'turbo', vmin = np.min(Gneural), vmax = np.max(Gneural))
# plt.colorbar()

In [ ]:
%%time
G = g[x,y]

plt.contourf(x,y,G, 50, cmap = 'turbo', vmin = np.min(G), vmax = np.max(G))
plt.colorbar()